In [1]:
!pip install torch torchvision

     |████████████████████████████████| 748.8 MB 12 kB/s s eta 0:00:01   |▊                               | 18.1 MB 1.7 MB/s eta 0:07:18     |████████                        | 186.7 MB 57.8 MB/s eta 0:00:10     |████████████████████▎           | 474.8 MB 40.0 MB/s eta 0:00:07     |█████████████████████████████▍  | 688.3 MB 57.3 MB/s eta 0:00:02
     |████████████████████████████████| 5.9 MB 45.2 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 45.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 44.3 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=493275 sha256=26096f1ed82c3825e34cb24b6b9967974963ce54cfa722e0b7ed920c0a0736d2
  Stored in directory: /root/.cache/pip/wheels/6e/9c/ed/4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
Successfully built future
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import torch
from torch import nn, optim, autograd
import numpy as np
import visdom
import random
from matplotlib import pyplot as plt

In [3]:
!python -m pip install --upgrade pip

     |████████████████████████████████| 1.5 MB 2.6 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2


In [4]:
!python -m pip install visdom

     |████████████████████████████████| 676 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 3.1 MB/s eta 0:00:01
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=668534 sha256=1ccc550c2ab9d565d65a76bd00c5c772a2d62bab554c4e7f3ab725b6056464ee
  Stored in directory: /root/.cache/pip/wheels/2d/cd/fb/005445070865d4e45365b2946ee88085a7392370f152cf371c
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=6623 sha256=83353dc45b982c077ff6c5979a220a788158f960b68570fad870e3cc80c8019e
  Stored in directory: /root/.cache/pip/wheels/55/79/ec/084a3a2e348d72852cc0c13c559c923c13ca54db86e699b681
Successfully built visdom torchfile


In [38]:
viz = visdom.Visdom()

Setting up a new session...
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/usr/local/lib/python3.6/dist-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/usr/local/lib/python3.6/dist-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 387, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/usr/lib/python3.6/http/client.py", line 1254, in request
    self._send_request(method, url, body,

Exception in user code:
------------------------------------------------------------


[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
Visdom python client failed to establish socket to get messages from the server. This feature is optional and can be disabled by initializing Visdom with `use_incoming_socket=False`, which will prevent waiting for this request to timeout.
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign reque

In [6]:
h_dim = 400
batchsz = 512

In [8]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, h_dim),
            nn.ReLU(True),
            nn.Linear(h_dim, h_dim),
            nn.ReLU(True),
            nn.Linear(h_dim, h_dim),
            nn.ReLU(True),
            nn.Linear(h_dim, 2),
        )
    def forward(self, z):
        output = self.net(z)
        return output

In [19]:
class Discriminator(nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(2, h_dim),
            nn.ReLU(True),
            nn.Linear(h_dim, h_dim),
            nn.ReLU(True),
            nn.Linear(h_dim, h_dim),
            nn.ReLU(True),
            nn.Linear(h_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.net(x)
        return output.view(-1)

In [20]:
def data_generator():
    scale = 2.
    centers = [
        (1,0),
        (-1,0),
        (0,1),
        (0,-1),
        (1./np.sqrt(2), 1./np.sqrt(2)),
        (1./np.sqrt(2), -1./np.sqrt(2)),
        (-1./np.sqrt(2), 1./np.sqrt(2)),
        (-1./np.sqrt(2), -1/np.sqrt(2))
    ]
    centers = [(scale*x, scale*y) for x, y in centers]
    
    while True:
        dataset = []
        for i in range(batchsz):
            point = np.random.randn(2)*0.02
            center = random.choice(centers)
            point[0] += center[0]
            point[1] += center[1]
            dataset.append(point)
        dataset = np.array(dataset).astype(np.float32)
        dataset /= 1.414
        yield dataset

In [35]:
def generate_image(D, G, xr, epoch):
    """
    Generates and saves a plot of the true distribution, the generator, and the
    critic.
    """
    N_POINTS = 128
    RANGE = 3
    plt.clf()

    points = np.zeros((N_POINTS, N_POINTS, 2), dtype='float32')
    points[:, :, 0] = np.linspace(-RANGE, RANGE, N_POINTS)[:, None]
    points[:, :, 1] = np.linspace(-RANGE, RANGE, N_POINTS)[None, :]
    points = points.reshape((-1, 2))
    # (16384, 2)
    # print('p:', points.shape)

    # draw contour
    with torch.no_grad():
        points = torch.Tensor(points).cuda() # [16384, 2]
        disc_map = D(points).cpu().numpy() # [16384]
    x = y = np.linspace(-RANGE, RANGE, N_POINTS)
    cs = plt.contour(x, y, disc_map.reshape((len(x), len(y))).transpose())
    plt.clabel(cs, inline=1, fontsize=10)
    # plt.colorbar()


    # draw samples
    with torch.no_grad():
        z = torch.randn(batchsz, 2).cuda() # [b, 2]
        samples = G(z).cpu().numpy() # [b, 2]
    plt.scatter(xr[:, 0], xr[:, 1], c='orange', marker='.')
    plt.scatter(samples[:, 0], samples[:, 1], c='green', marker='+')

    viz.matplot(plt, win='contour', opts=dict(title='p(x):%d'%epoch))

In [36]:
def main():
    torch.manual_seed(23)
    np.random.seed(23)
    data_iter = data_generator()
    x = next(data_iter)
    G = Generator().cuda()
    D = Discriminator().cuda()
    #print(G)
    #print(D)
    optim_G = optim.Adam(G.parameters(),lr=5e-4, betas=(0.5, 0.9))
    optim_D = optim.Adam(D.parameters(),lr=5e-4, betas=(0.5, 0.9))
    viz.line([[0,0]], [0], win='loss', opts=dict(title='loss',
                                                 legend=['D', 'G']))
    for epoch in range(50000):
        
        for _ in range(5):
            xr = next(data_iter)
            xr = torch.from_numpy(xr).cuda()
            predr = D(xr)
            lossr = -predr.mean()
            
            z = torch.randn(batchsz, 2).cuda()
            xf = G(z).detach()
            predf = D(xf)
            lossf = predf.mean()
            
            loss_D = lossr + lossf
            optim_D.zero_grad()
            loss_D.backward()
            optim_D.step()
        z = torch.randn(batchsz, 2).cuda()
        xf = G(z)
        predf = D(xf)
        loss_G = -predf.mean()
        optim_G.zero_grad()
        loss_G.backward()
        optim_G.step()
        
        if epoch %100 == 0:
            viz.line([[loss_D.item(), loss_G.item()]], [epoch], win='loss', update='append')
            generate_image(D, G, xr, epoch)
            print(loss_D.item(), loss_G.item())

In [37]:
main()

NameError: name 'viz' is not defined